In [1]:
# ! pip install kafka-python

IP: 192.168.1.39

In [7]:
import json
import socket
import time
import numpy as np
from datetime import datetime

import pandas as pd

In [3]:
sensor_map: dict = {
    1: "gps_position",
    3: "accelerometer",
    4: "gyroscope",
    5: "magnetic_field",
    81: "orientation",
    82: "linear_acceleration",
    83: "gravity",
    84: "rotation_vector",
    85: "pressure",
    86: "battery_temperature",
}

In [4]:
def get_payload(row: list) -> dict:
    row: list = list(map(float, row))
    dict_: dict = {}
    for key in sensor_map:
        if key in row:
            _index: int = row.index(key)
            if key not in (85, 86):
                dict_[sensor_map[key]] = row[_index + 1 : _index + 4]
            else:
                dict_[sensor_map[key]] = row[_index + 1 : _index + 2][0]
    return dict_

def produce_magnitude(listt):
    magg = np.sqrt(listt[0]**2 + listt[1]**2 + listt[2]**2)
    return magg

In [23]:
import pandas as pd

df = pd.read_csv("test.csv", index_col=0)
df.head(5)

,rotationRate,userAcceleration,act,id,weight,height,age,gender,trial
0,0.010253,0.006959,0.0,0.0,102.0,188.0,46.0,1.0,5.0
1,0.010920,0.010673,0.0,0.0,102.0,188.0,46.0,1.0,5.0
2,0.008377,0.007010,0.0,0.0,102.0,188.0,46.0,1.0,5.0
3,0.006555,0.014892,0.0,0.0,102.0,188.0,46.0,1.0,5.0
4,0.007724,0.013001,0.0,0.0,102.0,188.0,46.0,1.0,5.0


In [24]:
from sklearn.model_selection import train_test_split

Y = df['act']
X = df[['rotationRate', 'userAcceleration', 'weight', 'height', 'age', 'gender']]


X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=52)

print('Shape of X_train:', X_train.shape)
print('Shape of X_test:', X_test.shape)
print('Shape of Y_train:', Y_train.shape)
print('Shape of Y_test:', Y_test.shape)

Shape of X_train: (664716, 6)
Shape of X_test: (166179, 6)
Shape of Y_train: (664716,)
Shape of Y_test: (166179,)


In [25]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100)
model.fit(X_train, Y_train)

RandomForestClassifier()

In [26]:
Y_pred = model.predict(X_test)
Y_pred

array([1., 5., 5., ..., 1., 4., 4.])

In [27]:
from sklearn.metrics import classification_report
target_names = ['Walking', 'Walking Upstairs', 'Walking Downstairs', 'Sitting', 'Standing', 'Laying']

print(classification_report(Y_test, Y_pred, target_names=target_names))

                    precision    recall  f1-score   support

           Walking       0.83      0.81      0.82     46770
  Walking Upstairs       0.80      0.82      0.81     44994
Walking Downstairs       0.26      0.17      0.20      9988
           Sitting       0.35      0.27      0.31     11700
          Standing       0.58      0.69      0.63     31753
            Laying       0.71      0.71      0.71     20974

          accuracy                           0.70    166179
         macro avg       0.59      0.58      0.58    166179
      weighted avg       0.69      0.70      0.69    166179



In [29]:
labels = ['Walking', 'Walking Upstairs', 'Walking Downstairs', 'Sitting', 'Standing', 'Laying']

In [34]:
UDP_IP: str = ""
UDP_PORT: int = 5555

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sock.bind((UDP_IP, UDP_PORT))

i = 0
while True:
    data, addr = sock.recvfrom(1024)
    row: list = data.decode("utf-8").split(",")
    payload: dict = get_payload(row)
    payload["@timestamp"] = datetime.now().isoformat()
#     payload["sender"] = addr[0]
    payload["event_id"] = row[0].replace(".", "")

    if len(row) == 13:
#         print(i, " : ", "Len : ", len(row), ": ", row)
        
#         acc_x = row[2]
#         acc_y = row[3]
#         acc_z = row[4]
        
#         gyro_x = row[6]
#         gyro_y = row[7]
#         gyra_z = row[8]
        
#         mag_x = row[10]
#         mag_y = row[11]
#         mag_z = row[12]
        
        acc_mag = np.sqrt(float(row[2])**2 + float(row[3])**2 + float(row[4])**2)
        gyro_mag = np.sqrt(float(row[6])**2 + float(row[7])**2 + float(row[8])**2)
        magn_mag = np.sqrt(float(row[10])**2 + float(row[11])**2 + float(row[12])**2)
        
        data = [[gyro_mag, acc_mag, 74, 178, 24, 1]]
        out = model.predict(data)
        
        print(i, " : ","\nAcc Magnitude : ", acc_mag, "\nGyro Magnitude : ", gyro_mag, "\nMagneto Magnitude : ", magn_mag)
        print("Activity : ", labels[int(out)-1])
        i = i + 1

        

0  :  
Acc Magnitude :  9.963171784125777 
Gyro Magnitude :  0.0 
Magneto Magnitude :  40.30871668758508
Activity :  Laying
1  :  
Acc Magnitude :  9.956094866964657 
Gyro Magnitude :  0.0 
Magneto Magnitude :  40.075374396254865
Activity :  Laying
2  :  
Acc Magnitude :  9.970278531716152 
Gyro Magnitude :  0.001 
Magneto Magnitude :  40.50752604146543
Activity :  Laying
3  :  
Acc Magnitude :  9.952049587898967 
Gyro Magnitude :  0.0 
Magneto Magnitude :  39.22522072595641
Activity :  Laying
4  :  
Acc Magnitude :  9.972166966111228 
Gyro Magnitude :  0.001 
Magneto Magnitude :  39.58346459065957
Activity :  Laying
5  :  
Acc Magnitude :  9.956347623501301 
Gyro Magnitude :  0.0 
Magneto Magnitude :  39.702385696579995
Activity :  Laying
6  :  
Acc Magnitude :  8.11420106972954 
Gyro Magnitude :  3.2969393382347816 
Magneto Magnitude :  39.56822879533528
Activity :  Standing
7  :  
Acc Magnitude :  15.770632866185174 
Gyro Magnitude :  3.0573787792813634 
Magneto Magnitude :  38.6462

59  :  
Acc Magnitude :  16.968891713957042 
Gyro Magnitude :  2.33909063526833 
Magneto Magnitude :  38.84604094370493
Activity :  Standing
60  :  
Acc Magnitude :  9.970284950792529 
Gyro Magnitude :  0.002 
Magneto Magnitude :  40.896665707609955
Activity :  Laying
61  :  
Acc Magnitude :  9.990240537644727 
Gyro Magnitude :  0.001 
Magneto Magnitude :  41.444306738079234
Activity :  Laying
62  :  
Acc Magnitude :  9.978113950040859 
Gyro Magnitude :  0.0 
Magneto Magnitude :  40.653473566228016
Activity :  Laying
63  :  
Acc Magnitude :  9.99811407216381 
Gyro Magnitude :  0.0 
Magneto Magnitude :  40.71768373814994
Activity :  Laying
64  :  
Acc Magnitude :  9.98428595343703 
Gyro Magnitude :  0.001 
Magneto Magnitude :  40.66349672617936
Activity :  Laying
65  :  
Acc Magnitude :  9.97799874724386 
Gyro Magnitude :  0.0 
Magneto Magnitude :  41.20829056148774
Activity :  Laying
66  :  
Acc Magnitude :  9.980280707475115 
Gyro Magnitude :  0.0 
Magneto Magnitude :  40.444947830353

KeyboardInterrupt: 

In [ ]:
import gradio as gr

img = gr.inputs.Image()

title = "Face Mask - Spatial Domain Analysis(Smoothing)"

image1 = gr.inputs.Image(shape=(299, 299), type="numpy")
smoot = gr.inputs.Radio(['Median', 'Mode', "Average", "Weighted Avg."], label="Smoothing")

stat_out = gr.outputs.Textbox(type="auto", label="Statistics")
image2 = gr.outputs.Image(type="numpy", label="Output")

stat_out = gr.outputs.Textbox(type="auto", label="Comment")

gr.Interface(choice, inputs = [image1, smoot],   
             outputs = [image2, stat_out], 
             title=title,
             live=True).launch(share=True)